In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer

model_name = "xlm-roberta-base"

In [3]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_COPA")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [5]:
CONTEXT_COL = "premise"
QUESTION_COL = "question"
CHOICE_1_COL = "choice1"
CHOICE_2_COL = "choice2"


def preprocess_function(examples):
    """
    The preprocessing function needs to:
    1. Make two copies of the CONTEXT_COL field and combine each of them with QUESTION_COL to recreate how a sentence starts.
    2. Combine QUESTION_COL with each of the two possible choices.
    3. Flatten these two lists so you can tokenize them, and then unflatten them afterward so each example has a corresponding input_ids, attention_mask, and labels field.
    """

    question_headers = examples[QUESTION_COL]

    # Repeat each premise two times to go with the two choice possibilities.
    first_sentences = [[context] * 2 for context in examples[CONTEXT_COL]]
    # Grab all choices possible for each context.
    second_sentences = [
        [f"{header} {examples[choice][i]}" for choice in [CHOICE_1_COL, CHOICE_2_COL]] for i, header
        in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    return {k: [v[i: i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [6]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'question', 'choice1', 'choice2'], batched=True)

In [7]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [21]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [22]:
from transformers import set_seed

set_seed(42)

model = AutoModelForMultipleChoice.from_pretrained(model_name)
model.config.use_cache = False

Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from peft import LoraConfig, get_peft_model, TaskType

lora_alpha = 32
lora_rank_dropout = 0.1
lora_module_dropout = 0.0
lora_r = 16

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=lora_r,
    lora_alpha=lora_alpha,
    bias="none",
    base_model_name_or_path=model_name
)

In [24]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 590,593 || all params: 278,635,010 || trainable%: 0.2120


In [25]:
from transformers import TrainingArguments

new_model_name = "lora_fine_tuned_copa_XLMroberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [27]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=0.6970389080047608, metrics={'train_runtime': 21.7545, 'train_samples_per_second': 147.096, 'train_steps_per_second': 18.387, 'total_flos': 100513439953344.0, 'train_loss': 0.6970389080047608, 'epoch': 8.0})

In [28]:
trainer.evaluate()

{'eval_loss': 0.6928784251213074,
 'eval_accuracy': 0.59,
 'eval_f1': 0.5883995596036432,
 'eval_runtime': 0.304,
 'eval_samples_per_second': 328.947,
 'eval_steps_per_second': 42.763,
 'epoch': 8.0}

In [29]:
trainer.push_to_hub(new_model_name)

events.out.tfevents.1716313787.DESKTOP-22QTFDR.33008.3:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

events.out.tfevents.1716313765.DESKTOP-22QTFDR.33008.2:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/anzeo/lora_fine_tuned_copa_XLMroberta/commit/151e12d28c6a9a1148d940993038dc8a94e3dc2a', commit_message='lora_fine_tuned_copa_XLMroberta', commit_description='', oid='151e12d28c6a9a1148d940993038dc8a94e3dc2a', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
# Example
choice1 = "Naveličala sta se prepirov."
choice2 = "Izogibala sta se razgovoru o težavi."
prompt = "Odločila sta se skleniti kompromis."
question = "cause"

In [31]:
from peft import PeftModel, PeftConfig

# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "anzeo/" + new_model_name

config = PeftConfig.from_pretrained(adapter_name)
base_model = AutoModelForMultipleChoice.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name)

adapter_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [32]:
inputs = tokenizer([[prompt, f"{question} {choice1}"], [prompt, f"{question} {choice2}"]], return_tensors="pt",
                   padding=True)
labels = torch.tensor(0).unsqueeze(0)

outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
logits = outputs.logits

In [33]:
# Print prediction
logits.argmax().item()

0